In [ ]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

## Task 1.1

In [ ]:
def f2(x): #Här skapar vi en testfunktion 
    return x**2

def los2(x,alpha,beta,L): #Här definierar vi den exakta lösningen till ovanstående testfunktion
    return (x**4)/12 + (beta - alpha - (L**4)/12)*x/L + alpha


In [ ]:
def twopBVP(fvec, alpha, beta, L, N):
    deltaX = L/(N+1)

    #Här skapar vi T-matrisen med toeplits kommandot 
    first_row = np.array([-2, 1] + [0] * (N-2))
    T = la.toeplitz(first_row)
    T = T/(deltaX**2)

    #Här löser vi ut y och stoppar in randvärdena. 
    y = np.linalg.inv(T)@fvec
    y = np.insert(y, 0, alpha)
    y = np.append(y,beta)
    
    return  y #spottar ut en vektor innehållande y(i). 

def fvecgen(f,alpha, beta, L, N): #Tanken är att denna ska generera högerledet som man sedan stoppar in i twopBVP för att lösa. 
    deltaX = L/(N+1)

    Xvec = np.linspace(0,L,N+2)
    fvec = f(Xvec[1:N+1]) #Fyller fvec med funktionsvärdena F(x1) till F(xN-1)
    
    fvec[0] += -alpha/(deltaX**2) #Första och sista elementet i fvec fixar vi manuellt. 
    fvec[-1] += -beta/(deltaX**2)

    return fvec, Xvec

In [ ]:
f = f2
alpha = 4
beta = -100
N = 100
L = 10
fvec, Xvec = fvecgen(f, alpha, beta, L, N)
y = twopBVP(fvec, alpha, beta, L, N)

plt.plot(Xvec, y)
plt.grid()
plt.plot(Xvec, los2(Xvec,alpha,beta,L), '--')


In [ ]:
f = f2
Nvec = np.linspace(3,12,10).astype(int)
Nvec = 2**Nvec
errorvec = []
hvec = []
for N in Nvec:
    fvec, Xvec = fvecgen(f, alpha, beta, L, N)
    y = twopBVP(fvec, alpha, beta, L, N)
    deltaX = L/(N+1)
    errorvec.append(np.sqrt(deltaX)*la.norm(y - los2(Xvec,alpha,beta,L)))
 
    hvec.append(deltaX)

plt.figure()
plt.grid()
plt.plot(np.log(hvec), np.log(errorvec))
plt.plot(np.log(hvec), 2 * np.log(hvec) + np.log(errorvec[0]) - 2*np.log(hvec[0]), '--')
plt.xlabel('h')
plt.ylabel('Error')
plt.legend(['Our error', 'h^2'])
plt.title('Proving that the error is O(h^2)')


## Task 1.2

In [ ]:
E = 1.9e+11
L = 10 
alpha, beta = 0,0
N = 999 
q = -50000

def I(x):
    return 1e-3*(3-2*np.cos(np.pi*x/L)**12) 

Xvec = np.linspace(0,L,N+2)
Q = np.ones((N, 1)) * q
M = twopBVP(Q, alpha, beta,  L, N)

MEIvec = M[1:N+1]/(E*I(Xvec[1:N+1]))

U = twopBVP(MEIvec, alpha, beta,  L, N)  

plt.grid()
plt.plot(Xvec, U*1e3) #Förlänger med 1000 för att det ska ses lite tydligare 
plt.xlabel("x-coordinate (meter)")
plt.ylabel("Calculated Deflection in mm")
plt.title('Deflection as a function of position')

print("The deflection at the midpoint of the steelbeem is equal to " + str(1e3 * U[500]) + " mm")